<a href="https://colab.research.google.com/github/Next-Sunshine/TFF0318/blob/master/TestForTFFCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#环境测试
#@test {"skip":true}
!pip install --quiet --upgrade tensorflow_federated
%load_ext tensorboard

     |████████████████████████████████| 430kB 1.4MB/s 
     |████████████████████████████████| 2.2MB 44.2MB/s 
     |████████████████████████████████| 296kB 53.4MB/s 
     |████████████████████████████████| 20.0MB 215kB/s 
     |████████████████████████████████| 2.8MB 44.7MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 450kB 45.9MB/s 
     |████████████████████████████████| 3.9MB 50.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
import collections

import numpy as np
import random
import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.enable_v2_behavior()

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, world!')()

b'Hello, world!'

In [0]:
#装载数据集，实验使用EMNIST数据集
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [0]:
#此处得到客户端的总数K，因为每一轮随机选择C×K的向上取整
#根据FedAvg做的研究C取0.2可获得较好的性能，为了简化模型K这里取固定的数
K = len(emnist_train.client_ids)
C = 0.2

In [0]:
#NUM_CLIENTS在实验的时候应该是K×C
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

#预处理，将图片中的‘pixels'、‘label'分别表示成x和y
#将28×28的图像展平成784个元素，打乱顺序
def preprocess(dataset):
  #内部函数将像素和标签转换成x和y，并将像素展平
  def batch_format_fn(element):
    return collections.OrderedDict(
        #因为是灰度图所以通道数为1，如果是rgb图片则为3
        #因为CNN卷积层接收一个4d向量
        x = tf.reshape(element['pixels'], [-1,28,28,1]),
        y = tf.reshape(element['label'], [-1,1])
    )
  
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [0]:
#为指定用户创建联邦数据,接收训练集和用户id
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
    ]


In [0]:
#这里改成随机获得NUM_CLIENTS个用户，模拟联邦学习每轮的随机选择用户
 sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
# sample_clients #说明随机用户的id是取出来了的
# type(emnist_train.client_ids)  #client_ids是一个列表

federated_train_data = make_federated_data(emnist_train, sample_clients)

间隔界

In [151]:
'''
#！自己添加代码中，正在想办法把训练模型改成CNN
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        #print("kernel:",kernel_size)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
 
    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)
        #tf.nn.softmax()将原始输出归一化，且能凸显原始向量中最大的值                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output
        '''


'\n#！自己添加代码中，正在想办法把训练模型改成CNN\nclass CNN(tf.keras.Model):\n    def __init__(self):\n        super().__init__()\n        self.conv1 = tf.keras.layers.Conv2D(\n            filters=32,             # 卷积层神经元（卷积核）数目\n            kernel_size=[5, 5],     # 感受野大小\n            padding=\'same\',         # padding策略（vaild 或 same）\n            activation=tf.nn.relu   # 激活函数\n        )\n        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)\n        self.conv2 = tf.keras.layers.Conv2D(\n            filters=64,\n            kernel_size=[5, 5],\n            padding=\'same\',\n            activation=tf.nn.relu\n        )\n        #print("kernel:",kernel_size)\n        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)\n        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))\n        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)\n        self.dense2 = tf.keras.layers.Dense(units=10)\n \n    def call(self, inputs):\n        

间隔界


In [0]:
#创建一个变量集合来表示所有变量,包括model(weights+bias)以及metrics(num_examples,loss_sum,accuracy_sum)
# MnistVariables = collections.namedtuple(
#     'MnistVariables','weights bias num_examples loss_sum accuracy_sum'
# )
MnistVariables = collections.namedtuple(
    'MnistVariables','cnn_conv2d_kernel cnn_conv2d_bias cnn_conv2d_1_kernel cnn_conv2d_1_bias cnn_dense_kernel cnn_dense_bias cnn_dense_1_kernel cnn_dense_1_bias num_examples loss_sum accuracy_sum'
)

In [0]:
#创建变量并初始化
def create_mnist_variables():
  #核初始化函数，使用正态分布，先使用tf.XXX(初始参数)生成一个模型关键字C，再使用C(arg1)传参数进去
  kernel_initalizer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05)
  return MnistVariables(
      #第一层卷积在5×5的patch中算出32个特征，权重张量形状为[5,5,1,32]
      #前两个维度是patch的大小，接着是输入的通道数，最后是输出的通道数目
      #h_conv1.shape=[-1,28,28,32]，第一个pooling层将其变成[-1,14,14,32]     
      cnn_conv2d_kernel=tf.Variable(
          #权重初始化至关重要，不能全部初始化为0！！！
          initial_value=kernel_initalizer(shape=(5,5,1,32), dtype=tf.float32),
          name='cnn_conv2d_kernel',
          trainable=True),
      #32个输出通道，每个输出通道都有一个对应的偏置
      cnn_conv2d_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(32,)),
          name='cnn_conv2d_bias',
          trainable=True),
      #第二层卷积,64个过滤器，共享权重矩阵为32*5*5,h_conv2.shape=[-1,14,14,64]
      #第二个pooling层将其变成[-1,7,7,64]
      cnn_conv2d_1_kernel=tf.Variable(
          initial_value=kernel_initalizer(shape=(5,5,32,64), dtype=tf.float32),
          name='cnn_conv2d_1_kernel',
          trainable=True),
      #64个输出通道对应64个偏置
      cnn_conv2d_1_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(64,)),
          name='cnn_conv2d_1_bias',
          trainable=True),
      #全连接层，现在图片尺寸减小到7×7，加入一个有1024个神经元的全连接层用于处理整个图片
      #将池化层输出的张量reshape成一些7*7*64的向量，乘上权重加上偏置然后使用ReLU
      cnn_dense_kernel=tf.Variable(
          initial_value=kernel_initalizer(shape=(7*7*64,1024), dtype=tf.float32),
          name='cnn_dense_kernel',
          trainable=True),
      cnn_dense_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(1024,)),
          name='cnn_dense_bias',
          trainable=True),
      #这里以后可以加一个Dropout
      cnn_dense_1_kernel=tf.Variable(
          initial_value=kernel_initalizer(shape=(1024,10), dtype=tf.float32),
          name='cnn_dense_1_kernel',
          trainable=True),
      cnn_dense_1_bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(10,)),
          name='cnn_dense_1_bias',
          trainable=True),
      num_examples=tf.Variable(0.0, name='num_examples', trainable=False),
      loss_sum=tf.Variable(0.0, name='loss_sum', trainable=False),
      accuracy_sum=tf.Variable(0.0, name='accuracy_sum', trainable=False)
  )

In [0]:
#自定义前向传播函数
def mnist_forward_pass(variables, batch):
  _input_ = batch['x']  #取训练图片

  #神经网络
  #第一层卷积
  conv = tf.nn.conv2d(_input_, variables.cnn_conv2d_kernel, strides=[1,1,1,1], padding='SAME')                  # [batch_size, 28, 28, 32]
  pre_activation = tf.nn.bias_add(conv, variables.cnn_conv2d_bias)
  conv1 = tf.nn.relu(pre_activation, name='conv1')

  #第一层卷积层的池化
  pool1 = tf.nn.max_pool2d(conv1, ksize=[2, 2], strides=2, padding='SAME')

  #第二层卷积
  conv = tf.nn.conv2d(pool1, variables.cnn_conv2d_1_kernel, strides=[1,1,1,1], padding='SAME')
  pre_activation = tf.nn.bias_add(conv, variables.cnn_conv2d_1_bias)
  conv2 = tf.nn.relu(pre_activation, name='conv2')

  #第二层卷积的池化
  pool2 = tf.nn.max_pool2d(conv2, ksize=[2,2], strides=2, padding='SAME')
  print("pool2",pool2)
  
  #flatten
  flatten = tf.reshape(pool2, shape=(-1,7*7*64))
  print("flatten over")

  #全连接层1
  dense1 = tf.nn.relu(tf.matmul(flatten, variables.cnn_dense_kernel) + variables.cnn_dense_bias)
  print("dense1 over")

  #全连接层2
  dense2 = tf.matmul(dense1, variables.cnn_dense_1_kernel) + variables.cnn_dense_1_bias
  print("dense2 over")

  y_pred = tf.nn.softmax(dense2)
  print("y_pred",y_pred)

  #计算损失
  flat_labels = tf.reshape(batch['y'], [-1])
  loss = -tf.reduce_mean(
      tf.reduce_sum(tf.one_hot(flat_labels, 10) * tf.math.log(y_pred), axis=[1]))
  
  
  #计算准确率
  predictions = tf.cast(tf.argmax(y_pred,1), tf.int32)
  accuracy = tf.reduce_mean(
      tf.cast(tf.equal(predictions, flat_labels), tf.float32))
  
  #样本数
  num_examples = tf.cast(tf.size(batch['y']), tf.float32)

  #更新样本数、损失和、精度和,每一批都考虑了自己的权重
  variables.num_examples.assign_add(num_examples)
  variables.loss_sum.assign_add(loss * num_examples)
  variables.accuracy_sum.assign_add(accuracy * num_examples)

  
  return loss, predictions

In [0]:
#计算本地用户的metrics度量
def get_local_mnist_metrics(variables):
  return collections.OrderedDict(
      num_examples=variables.num_examples,
      loss=variables.loss_sum / variables.num_examples,
      accuracy=variables.accuracy_sum / variables.num_examples
  )

In [0]:
#集合每个设备发出的本地度量
@tff.federated_computation
def aggregate_mnist_metrics_across_clients(metrics):
  return collections.OrderedDict(
      num_examples=tff.federated_sum(metrics.num_examples),
      loss=tff.federated_mean(metrics.loss, metrics.num_examples),
      accuracy=tff.federated_mean(metrics.accuracy, metrics.num_examples)
  )

In [0]:
#自定义模型，创建tff.learning.model实例
class MnistModel(tff.learning.Model):

  def __init__(self):
    self._variables = create_mnist_variables()

  #所有的“tf.Variables”都应该在“__init__”中引入
  @property
  def trainable_variables(self):
    #return [self._variables.weights, self._variables.bias]
    return [self._variables.cnn_conv2d_kernel,
        self._variables.cnn_conv2d_bias,
        self._variables.cnn_conv2d_1_kernel,
        self._variables.cnn_conv2d_1_bias,
        self._variables.cnn_dense_kernel,
        self._variables.cnn_dense_bias,
        self._variables.cnn_dense_1_kernel,
        self._variables.cnn_dense_1_bias
        ]
  
  @property
  def non_trainable_variables(self):
    return []
  
  @property
  def local_variables(self):
    return [self._variables.num_examples, self._variables.loss_sum,
         self._variables.accuracy_sum]
  
  @property
  def input_spec(self):
    return collections.OrderedDict(
        x=tf.TensorSpec([None, 28,28,1], tf.float32),
        y=tf.TensorSpec([None, 1], tf.int32)
    )

  @tf.function
  def forward_pass(self, batch, training=True):
    del training
    loss, predictions = mnist_forward_pass(self._variables, batch)
    #取出样本数
    num_examples = tf.shape(batch['x'])[0]
    return tff.learning.BatchOutput(
        loss=loss, predictions=predictions, num_examples=num_examples)
    
  @tf.function
  def report_local_outputs(self):
    return get_local_mnist_metrics(self._variables)
  
  @property
  def federated_output_computation(self):
    return aggregate_mnist_metrics_across_clients

In [159]:
#创建迭代器执行联合平均的迭代过程
iterative_process = tff.learning.build_federated_averaging_process(
    MnistModel,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02)
)

pool2 Tensor("MaxPool2d_1:0", shape=(None, 7, 7, 64), dtype=float32)
flatten over
dense1 over
dense2 over
y_pred Tensor("Softmax:0", shape=(None, 10), dtype=float32)


In [0]:
#获得初始状态
state = iterative_process.initialize()

In [161]:
#查看第一轮训练
sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
federated_train_data = make_federated_data(emnist_train, sample_clients)
state, metrics = iterative_process.next(state, federated_train_data)
print('round 1, metrics={}'.format(metrics))

round 1, metrics=<num_examples=4485.0,loss=2.3019957542419434,accuracy=0.11571906507015228>


In [163]:
#计算更多轮
NUM_ROUNDS = 20
for round_num in range(2,NUM_ROUNDS):
  #随机选择NUM_CLIENTS个用户
  sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
  federated_train_data = make_federated_data(emnist_train, sample_clients)

  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=<num_examples=4860.0,loss=1.665802001953125,accuracy=0.565432071685791>
round  3, metrics=<num_examples=5255.0,loss=1.3768004179000854,accuracy=0.6319695711135864>
round  4, metrics=<num_examples=4800.0,loss=1.19942045211792,accuracy=0.6731250286102295>
round  5, metrics=<num_examples=5180.0,loss=0.822767972946167,accuracy=0.7830115556716919>
round  6, metrics=<num_examples=5240.0,loss=0.6395965814590454,accuracy=0.8198473453521729>
round  7, metrics=<num_examples=5140.0,loss=0.6477411985397339,accuracy=0.8202334642410278>
round  8, metrics=<num_examples=5165.0,loss=0.4296082854270935,accuracy=0.8913843035697937>
round  9, metrics=<num_examples=4980.0,loss=0.4372885525226593,accuracy=0.8734939694404602>
round 10, metrics=<num_examples=5430.0,loss=0.3003905415534973,accuracy=0.9250460267066956>
round 11, metrics=<num_examples=4920.0,loss=0.3301681578159332,accuracy=0.9132114052772522>
round 12, metrics=<num_examples=5345.0,loss=0.36047056317329407,accuracy=0.8963517546

In [0]:
'''
#使用tensorboard可视化
#使用Tensorboard可视化这些联邦计算的度量
#创建目录和相应的摘要编写器
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()

#!!!太坑了，我以为@test是无关紧要的东西，emmm，就省略了，没想到就凉凉
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    #随机选择用户
    sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
    federated_train_data = make_federated_data(emnist_train, sample_clients)

    state, metrics = iterative_process.next(state, federated_train_data)

    print('round {:2d}, metrics={}'.format(round_num, metrics))

    for name, value in metrics._asdict().items():
      tf.summary.scalar(name, value, step=round_num)

#这个tensorBoard像有猫病一样，昨天还可以今天来突然就不行了
#@test {"skip":true}
#%load_ext tensorboard
%tensorboard --logdir /tmp/logs/scalars/ --port=0
'''